In [8]:
import math
import re
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
#step 1: sample document setup
#simple documents
sample_documents = [
    "The cat sat on the mat",
    "The dog run in the park",
    "Cats and dogs are pets",
    "I love my pet cat",
    "The park has many trees"
]

print("Our sample documents")
for i, doc in enumerate(sample_documents):
    print(f"Doc{i}:{doc}")

Our sample documents
Doc0:The cat sat on the mat
Doc1:The dog run in the park
Doc2:Cats and dogs are pets
Doc3:I love my pet cat
Doc4:The park has many trees


In [10]:
#step 2: text pre-processing
def preprocess_text(text):
    """
    Clean and tokenize text
    - Convert to lowercase
    - Remove punctuation
    - split into words
    """
    #convert to lowercase
    text = text.lower()

    #remove punctuation and splits into words
    words = re.findall(r'\b[a-z]+\b', text)

    return words

# test pre-processing
print("\n" + "="*50)
print("Step 2: Text Preprocessing")
print("="*50)

test_doc = sample_documents[0]
processed = preprocess_text(test_doc)
print(f"Original:{test_doc}")
print(f"Processed: {processed}" )

#process all documents
processed_docs = [preprocess_text(doc) for doc in sample_documents]
print(f"\nAll processed documents:")
for i, doc in enumerate(processed_docs):
    print(f"Doc {i}: {doc}")



Step 2: Text Preprocessing
Original:The cat sat on the mat
Processed: ['the', 'cat', 'sat', 'on', 'the', 'mat']

All processed documents:
Doc 0: ['the', 'cat', 'sat', 'on', 'the', 'mat']
Doc 1: ['the', 'dog', 'run', 'in', 'the', 'park']
Doc 2: ['cats', 'and', 'dogs', 'are', 'pets']
Doc 3: ['i', 'love', 'my', 'pet', 'cat']
Doc 4: ['the', 'park', 'has', 'many', 'trees']


In [11]:
#step 3: build vocabulary
def build_vocabulary(processed_docs):
    """
    Create a vocabulary/unique words from all documents
    """
    vocab = set()
    for doc in processed_docs:
        vocab.update(doc)
    return sorted(list(vocab)) 

print("\n" + "="*50)
print("Step 3: Build vocabulary")
print("="*50)

vocabulary = build_vocabulary(processed_docs)
print(f"Vocabulary ({len(vocabulary)}words): {vocabulary}")


Step 3: Build vocabulary
Vocabulary (21words): ['and', 'are', 'cat', 'cats', 'dog', 'dogs', 'has', 'i', 'in', 'love', 'many', 'mat', 'my', 'on', 'park', 'pet', 'pets', 'run', 'sat', 'the', 'trees']


In [12]:
#step 4 : calculate term frequency
def calculate_tf(doc_words, vocabulary):
    """
    calculate term frequency for a single document
    TF = (word count in document) / (total words in document)
    """
    tf_dict = {}
    doc_length = len(doc_words)
    word_counts = Counter(doc_words)

    for word in vocabulary:
        tf_dict[word] = word_counts[word] / doc_length

    return tf_dict

print("\n" + "="*50)
print("step 4: calculate term frequency (TF)")
print("="*50)

tf_docs = []
for i, doc in enumerate(processed_docs):
    tf = calculate_tf(doc,vocabulary)
    tf_docs.append(tf)
    print(f"\nDoc {i} TF Scores:")

    #show only non-zero TF Scores for clarity

    non_zero_tf = {word: score for word, score in tf.items() if score > 0}
    for word, score in non_zero_tf.items():
        print(f"{word}:{score:.3f}")


step 4: calculate term frequency (TF)

Doc 0 TF Scores:
cat:0.167
mat:0.167
on:0.167
sat:0.167
the:0.333

Doc 1 TF Scores:
dog:0.167
in:0.167
park:0.167
run:0.167
the:0.333

Doc 2 TF Scores:
and:0.200
are:0.200
cats:0.200
dogs:0.200
pets:0.200

Doc 3 TF Scores:
cat:0.200
i:0.200
love:0.200
my:0.200
pet:0.200

Doc 4 TF Scores:
has:0.200
many:0.200
park:0.200
the:0.200
trees:0.200


In [13]:
#step 5: calculate inverse document frequency
def calculate_idf(processed_docs, vocabulary):
    """
    Calculate IDF for each word in vocabulary
    IDF = log(total documents / documents containing the word)
    """

    idf_dict = {}
    total_docs = len(processed_docs)

    for word in vocabulary:
        # count how many documents contain this word
        docs_containing_word = sum(1 for doc in processed_docs if word in doc)

        # calculate idf
        idf_dict[word] = math.log(total_docs / docs_containing_word)

    return idf_dict

print("\n" + "="*50)
print("step 5: calculating inverse document frequency(IDF)")
print("="*50)

idf_scores = calculate_idf(processed_docs, vocabulary)
print("IDF Scores:")
for word, score in idf_scores.items():
    docs_with_word = sum(1 for doc in processed_docs if word in doc)
    print(f"{word}: {score:.3f} (appears in {docs_with_word}/{len(processed_docs)} docs)")


step 5: calculating inverse document frequency(IDF)
IDF Scores:
and: 1.609 (appears in 1/5 docs)
are: 1.609 (appears in 1/5 docs)
cat: 0.916 (appears in 2/5 docs)
cats: 1.609 (appears in 1/5 docs)
dog: 1.609 (appears in 1/5 docs)
dogs: 1.609 (appears in 1/5 docs)
has: 1.609 (appears in 1/5 docs)
i: 1.609 (appears in 1/5 docs)
in: 1.609 (appears in 1/5 docs)
love: 1.609 (appears in 1/5 docs)
many: 1.609 (appears in 1/5 docs)
mat: 1.609 (appears in 1/5 docs)
my: 1.609 (appears in 1/5 docs)
on: 1.609 (appears in 1/5 docs)
park: 0.916 (appears in 2/5 docs)
pet: 1.609 (appears in 1/5 docs)
pets: 1.609 (appears in 1/5 docs)
run: 1.609 (appears in 1/5 docs)
sat: 1.609 (appears in 1/5 docs)
the: 0.511 (appears in 3/5 docs)
trees: 1.609 (appears in 1/5 docs)


In [14]:
#step 6: calculate tf-idf scores
def calculate_tfidf(tf_docs, idf_scores, vocabulary):
    """
    Calculate TF-IDF = TF x IDF for all documents
    """
    tfidf_docs = []

    for tf_doc in tf_docs:
        tfidf_dict = {}
        
        for word in vocabulary:
            tfidf_dict[word] = tf_doc[word] * idf_scores[word]
        tfidf_docs.append(tfidf_dict)

    return tfidf_docs

print("\n" + "="*50)
print("Step 6: Calculate TF-IDF scores")
print("="*50)

tfidf_docs = calculate_tfidf(tf_docs,idf_scores, vocabulary)

# Display tf-idf scores in a table
tfidf_df = pd.DataFrame(tfidf_docs)
tfidf_df.index = [f"Doc{i}" for i in range(len(sample_documents))]
print("TF-IDF Matrix:")
print(tfidf_df.round(3))


Step 6: Calculate TF-IDF scores
TF-IDF Matrix:
        and    are    cat   cats    dog   dogs    has      i     in   love  \
Doc0  0.000  0.000  0.153  0.000  0.000  0.000  0.000  0.000  0.000  0.000   
Doc1  0.000  0.000  0.000  0.000  0.268  0.000  0.000  0.000  0.268  0.000   
Doc2  0.322  0.322  0.000  0.322  0.000  0.322  0.000  0.000  0.000  0.000   
Doc3  0.000  0.000  0.183  0.000  0.000  0.000  0.000  0.322  0.000  0.322   
Doc4  0.000  0.000  0.000  0.000  0.000  0.000  0.322  0.000  0.000  0.000   

      ...    mat     my     on   park    pet   pets    run    sat    the  \
Doc0  ...  0.268  0.000  0.268  0.000  0.000  0.000  0.000  0.268  0.170   
Doc1  ...  0.000  0.000  0.000  0.153  0.000  0.000  0.268  0.000  0.170   
Doc2  ...  0.000  0.000  0.000  0.000  0.000  0.322  0.000  0.000  0.000   
Doc3  ...  0.000  0.322  0.000  0.000  0.322  0.000  0.000  0.000  0.000   
Doc4  ...  0.000  0.000  0.000  0.183  0.000  0.000  0.000  0.000  0.102   

      trees  
Doc0  0.000 

In [15]:
# step 7: Document similarity search

def cosine_similarity_manual(vec1, vec2):
    """
    calculate cosine similarity between two vectors
    """

    dot_product = sum(a * b for a, b in zip (vec1, vec2) )
    magnitude1 = math.sqrt(sum(a * a for a in vec1))
    magnitude2 = math.sqrt(sum(b * b for b in vec2))

    if magnitude1 == 0 or magnitude2 == 0:
        return 0

    return dot_product / (magnitude1 * magnitude2)

def search_documents(query, sample_documents, tfidf_docs, vocabulary, idf_scores):
    """
    Search for most similar documents to a query
    """

    # process query
    query_words = preprocess_text(query)

    # calculate query TF-IDF
    query_tf = calculate_tf(query_words, vocabulary)
    query_tfidf = {word:query_tf[word] * idf_scores[word] for word in vocabulary}

    # convert to vectors
    query_vector = [query_tfidf[word] for word in vocabulary]

    # calculate similarities
    similarities = []
    for i, doc_tfidf in enumerate(tfidf_docs):
        doc_vector = [doc_tfidf[word] for word in vocabulary]
        similarity = cosine_similarity_manual(query_vector, doc_vector)
        similarities.append((i,similarity))

    # sort by similarity (highest first)
    similarities.sort(key = lambda x: x[1], reverse = True)

    return similarities

print("\n" + "="*50)
print("STEP 7: Document Similarity Search")
print("="*50)

# Test search
test_query = "cats and dogs"
print(f"Search query: '{test_query}'")
print("\nSearch results (most similar first):")

results = search_documents(test_query, sample_documents, tfidf_docs, vocabulary, idf_scores)

for rank, (doc_idx, similarity) in enumerate(results[:3],1):
    print(f"{rank}. Doc{doc_idx}: '{sample_documents[doc_idx]}' (similarity:{similarity:.3f})")


STEP 7: Document Similarity Search
Search query: 'cats and dogs'

Search results (most similar first):
1. Doc2: 'Cats and dogs are pets' (similarity:0.775)
2. Doc0: 'The cat sat on the mat' (similarity:0.000)
3. Doc1: 'The dog run in the park' (similarity:0.000)


In [16]:
# step 8: Compare with scikit-learn

print("\n" + "="*50)
print("STEP 8: Validation with Scikit-Learn")
print("="*50)

# use scikit-learn for comparison
vectorizer = TfidfVectorizer(lowercase = True, token_pattern = r'\b[a-z]+\b')
sklearn_tfidf = vectorizer.fit_transform(sample_documents)
sklearn_feature_names = vectorizer.get_feature_names_out()

#convert to dataframe for comparison
sklearn_df = pd.DataFrame(sklearn_tfidf.toarray(), 
                          columns=sklearn_feature_names, 
                          index = [f"Doc_{i}" for i in range(len(sample_documents))])

print("Scikit-Learn TF-IDF Matrix:")
print(sklearn_df.round(3))

print(f"\nComparison:")
print(f"Our Vocbulary size: {len(vocabulary)}")
print(f"Scikit-learn vocabulary size: {len(sklearn_feature_names)}")
print(f"Vocabularies match: {set(vocabulary) == set(sklearn_feature_names)}")


STEP 8: Validation with Scikit-Learn
Scikit-Learn TF-IDF Matrix:
         and    are    cat   cats    dog   dogs    has      i     in   love  \
Doc_0  0.000  0.000  0.346  0.000  0.000  0.000  0.000  0.000  0.000  0.000   
Doc_1  0.000  0.000  0.000  0.000  0.429  0.000  0.000  0.000  0.429  0.000   
Doc_2  0.447  0.447  0.000  0.447  0.000  0.447  0.000  0.000  0.000  0.000   
Doc_3  0.000  0.000  0.374  0.000  0.000  0.000  0.000  0.464  0.000  0.464   
Doc_4  0.000  0.000  0.000  0.000  0.000  0.000  0.494  0.000  0.000  0.000   

       ...    mat     my     on   park    pet   pets    run    sat    the  \
Doc_0  ...  0.429  0.000  0.429  0.000  0.000  0.000  0.000  0.429  0.574   
Doc_1  ...  0.000  0.000  0.000  0.346  0.000  0.000  0.429  0.000  0.574   
Doc_2  ...  0.000  0.000  0.000  0.000  0.000  0.447  0.000  0.000  0.000   
Doc_3  ...  0.000  0.464  0.000  0.000  0.464  0.000  0.000  0.000  0.000   
Doc_4  ...  0.000  0.000  0.000  0.398  0.000  0.000  0.000  0.000  0.331 

In [17]:
# step 9: complete tf-idf class


class SimpleTFIDF:
    """
    A simple tfidf implementation from scratch
    """

    def __init__(self):
        self.vocabulary = []
        self.idf_scores = {}
        self.documents  = []

    def fit(self,documents):
        """Train the tf-idf model on documents"""

        self.documents = documents
        processed_docs = [preprocess_text(doc) for doc in documents]
        self.vocabulary = build_vocabulary(processed_docs)
        self.idf_scores = calculate_idf(processed_docs, self.vocabulary)

        return self

    def transform(self,documents):
        """transform documents to tf-idf vectors"""

        processed_docs = [preprocess_text(doc) for doc in documents]
        tf_docs = [calculate_tf(doc, self.vocabulary) for doc in processed_docs]
        tfidf_docs = calculate_tfidf(tf_docs, self.idf_scores, self.vocabulary)

        """convert to matrix form"""

        matrix = []
        for tfidf_doc in tfidf_docs:
            vector = [tfidf_doc[word] for word in self.vocabulary]
            matrix.append(vector)

        return np.array(matrix)

    def fit_transform(self, documents):
        """fit and transform in one step"""
        return self.fit(documents).transform(documents)

    def search(self, query, top_k =3):
        """search for most similar documents"""
        query_vector = self.transform([query])[0]
        doc_vectors = self.transform(self.documents)

        similarities = []
        for i, doc_vector in enumerate(doc_vectors):
            similarity = cosine_similarity_manual(query_vector, doc_vector)
            similarities.append((i,similarity, self.documents[i]))

        similarities.sort(key=lambda x:x[1], reverse=True)
        return similarities[:top_k]

print("\n" + "="*50)
print("STEP 9: Complete TF-IDF Class Usage")
print("="*50)

# test the complete tf-idf class
tfidf_model = SimpleTFIDF()
tfidf_matrix = tfidf_model.fit_transform(sample_documents)

print("TF-IDF Matrix shape:", tfidf_matrix.shape)
print("Vocabulary size:", len(tfidf_model.vocabulary))

# test search

print(f"\nSearch results for '{test_query}':")
search_results = tfidf_model.search(test_query)
for rank, (doc_idx, similarity, document) in enumerate (search_results, 1):
    print(f"{rank}. '{document}' (similarity:{similarity:.3f})")
        


STEP 9: Complete TF-IDF Class Usage
TF-IDF Matrix shape: (5, 21)
Vocabulary size: 21

Search results for 'cats and dogs':
1. 'Cats and dogs are pets' (similarity:0.775)
2. 'The cat sat on the mat' (similarity:0.000)
3. 'The dog run in the park' (similarity:0.000)
